- instance method 呼ばれる仕組みを見ていくよ
    - selfってなんなんだろう？
    - どう渡されてるのだろう？

In [2]:
class Human:
    def __init__(self, name):
        self.name = name
    
    def greet(self, your_name):
        print('hello {0}, i am {1}.'.format(your_name, self.name))
        
def greet_func(your_name, my_name):
        print('hello {0}, i am {1}.'.format(your_name, my_name))

In [3]:
h = Human('taro')
h.greet('jiro')

hello jiro, i am taro.


In [4]:
greet_func('jiro', 'taro')

hello jiro, i am taro.


- コードとして引数いくつ取る想定なの？を見てみよう

In [6]:
h.greet.__code__.co_argcount

2

In [7]:
greet_func.__code__.co_argcount

2

- どちらも2引数であるようだ
    - でも 実際に利用する時 `greet` は `'jiro'` しか受けてないよね
    - もう一個の引数は？
- self　として渡されてるモノが重要

- ポイントは descriptor
    - https://docs.python.jp/3/howto/descriptor.html?highlight=%E3%83%87%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%82%BF#invoking-descriptors
    
- こんなことが起きてる

```python
b.x
type(b).__dict__['x'].__get__(b, type(b)) 
```

In [9]:
# これが同一ってことか
h.greet('jiro')
type(h).__dict__['greet'].__get__(h, type(h))('jiro')
type(h).__dict__['greet'](h, 'jiro')

hello jiro, i am taro.
hello jiro, i am taro.
hello jiro, i am taro.


In [13]:
h.greet

<bound method Human.greet of <__main__.Human object at 0x10bbb0470>>

In [14]:
Human.greet

<function __main__.Human.greet>

- bound method は 第一引数に object 自体が 部分適用される関数 とみて良さそう
- bound method を decorate すると どうなるんだっけ？を考えるといいのかも
  - たぶん ただの function になって boundされるというcontextがなくなる，という話なのかな？
  - @で書くといい感じだけど， f = deco(f) だと 駄目，みたいなイメージ
- class 内外で見た目変わるのかなーとかそういうメタな話．．．


In [22]:
def deco(f):
    def decorated(*args, **kwargs):
        print('--begin--')
        result = f(*args, **kwargs)
        print('--end--')
        return result
    return decorated

class Human2:
    def __init__(self, name):
        self.name = name
    
    def greet(self, your_name):
        print('hello {0}, i am {1}.'.format(your_name, self.name))
        
    @deco
    def auto_deco_greet(self, your_name):
        print('hello {0}, i am {1}.'.format(your_name, self.name))
    
    def manual_deco_greet(self, your_name):
        f =deco(self.greet)
        print(f)
        f(your_name)

    @property
    def manual_deco_greet2(self):
        return [deco(f) for f in [self.greet]]
 
    def manual_deco_greet2_apply(self, your_name):
        self.manual_deco_greet2[0](your_name)
    
    
h2 = Human2('taro')
h2.greet('jiro')
h2.auto_deco_greet('jiro')
h2.manual_deco_greet('jiro')
print(h2.manual_deco_greet2)
h2.manual_deco_greet2[0]('jiro')
h2.manual_deco_greet2_apply('jiro')

hello jiro, i am taro.
--begin--
hello jiro, i am taro.
--end--
<function deco.<locals>.decorated at 0x10c418950>
--begin--
hello jiro, i am taro.
--end--
[<function deco.<locals>.decorated at 0x10c4188c8>]
--begin--
hello jiro, i am taro.
--end--
--begin--
hello jiro, i am taro.
--end--


ふつーにうごくやーん